<a href="https://colab.research.google.com/github/calmrocks/master-machine-learning-engineer/blob/main/GenAI/PromptEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering Techniques Demonstration
This notebook demonstrates various prompt engineering techniques using Hugging Face transformers. We'll use a medium-sized language model to showcase different prompting strategies.

In [1]:

import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import time

## Model Setup
We'll use a medium-sized language model (OPT-1.3B) that balances performance and resource usage. This model can run effectively on Google Colab's GPU runtime.

In [2]:
# Model Setup and Utilities
def setup_model(model_name="facebook/opt-1.3b", use_4bit=False):
    try:
        if use_4bit:
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                load_in_4bit=True,
                torch_dtype=torch.bfloat16,
                device_map="auto"
            )
            generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
        else:
            generator = pipeline(
                "text-generation",
                model=model_name,
                torch_dtype=torch.bfloat16,
                device_map="auto"
            )
        return generator
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def generate_response(generator, prompt, max_length=200, temperature=0.7):
    response = generator(
        prompt,
        max_length=max_length,
        temperature=temperature,
        num_return_sequences=1,
        pad_token_id=generator.tokenizer.eos_token_id
    )
    return response[0]['generated_text']

# Initialize model
generator = setup_model("facebook/opt-1.3b")  # or choose another model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Device set to use cpu


## 1. Zero-shot Prompting
Zero-shot prompting is the simplest form of prompting where we directly ask the model to perform a task without providing any examples. This technique tests the model's ability to understand and perform tasks based solely on instructions.

Key characteristics:
- No examples provided
- Relies on model's pre-trained knowledge
- Simplest to implement but may not always give optimal results

In [ ]:

print("Zero-shot Prompting Example")
print("-" * 50)

zero_shot_prompt = """Classify the sentiment of this text as positive, negative, or neutral:
Text: "I absolutely love this new phone! It's amazing!"
Sentiment:"""

response = generate_response(generator, zero_shot_prompt)
print(response)